# Project 1
By Alexander Hatle

In [1]:
import numpy as np
import time
np.random.seed(40)
#<img src="">

## Task 1
### a)

<img src="pictures/Screenshot 2022-09-19 at 11.07.35.png">

$$A = \begin{pmatrix}0 & 0 & 0 \\ 0 & 0 & 0\end{pmatrix} \quad \text{and} \quad \vec y = \begin{pmatrix}1  \\  1\end{pmatrix}$$

<img src="pictures/Screenshot 2022-09-19 at 11.12.01.png">

$$A = \begin{pmatrix}0 & 0 & 0 \\ 0 & 0 & 0\end{pmatrix} \quad \text{and} \quad \vec y = \begin{pmatrix}0  \\  0\end{pmatrix}$$

### b)

<img src="pictures/Screenshot 2022-09-19 at 11.14.49.png">

We know that a matrix A is symmetric if $A^T=A$ 
$$(A^{T}A)^{T}=A^{T}(A^{T})^{T} = A^{T}A \Rightarrow \text{Symmetric}$$
$$(AA^{T})^{T}=(A^{T})^{T} A^{T}= AA^{T} \Rightarrow \text{Symmetric}$$
A matrix is positive semi-definite if it satisfies
$$\vec x^{T}A\vec x \ge 0, \qquad \text{for all } x\in \mathbb{R}^{n}$$
$$\vec x^{T}A^{T}A\vec x = (A\vec x)^{T }A\vec x= ||A\vec x||^{2} \ge 0$$
$$\vec x^{T}AA^{T}\vec x=(A^{T}\vec x)^{T}A^{T}\vec x=||A^{T}\vec x||^{2} \ge 0$$
So both $A^{T}A$ and $AA^{T}$ are symmetric positive semi-definite.

In [6]:
#TODO 1b) invertibility
# mxm invertible, nxn not invertible

### c)

<img src="pictures/Screenshot 2022-09-19 at 11.35.33.png">

$$\begin{align*}
\mathcal{L}(\vec x, \vec \lambda) &= \vec x^{T}\vec x+\vec \lambda(A\vec x -\vec y)\\
&=\sum\limits_{i=1}^{n}\vec x_{i}^{2} + \lambda_{i}\left(\sum\limits_{j=1}^{m}a_{ij}x_{j}-y_{i}\right)\\
\frac{\partial \mathcal{L}}{\partial x_{k}}&=2x_{k}+\lambda_{k}\sum\limits_{j=1}^{m}a_{kj}\\
&=2x_{k}+\sum\limits_{j=1}^{m}a_{kj}\lambda_{k}\\
\nabla_{x}\mathcal{L}&=2\vec x +A^{T}\vec \lambda=0&\qquad\text{(1)}\\
\frac{\partial\mathcal{L}}{\partial \lambda_{k}}&=\sum\limits_{j=1}^{m}a_{kj}x_{j}-y_{k}\\
\nabla_{\lambda}\mathcal{L}&=A\vec x-\vec y=0 &\qquad\text{(2)}\\
\vec x&=- \frac{1}{2}A^{T}\vec \lambda\qquad&\text{rewriting (1)}\\
 A\left(- \frac{1}{2}A^{T}\vec \lambda\right)&=- \frac{1}{2}AA^{T}\vec \lambda=\vec y&\text{(1) in (2)}\quad\\
\vec \lambda&=-2(AA^{T})^{-1}\vec y\\
2\vec x +A^{T}\vec\lambda&=2\vec x-2A^{T}(AA^{T})^{-1}\vec y=0 &\text{in (2)}\\
\vec x &=A^{T}(AA^{T})^{-1}\vec y \qquad\square
\end{align*}$$

### d)

### e)

### f)
#### 1)

#### 2)

## Task 2
### a)

### b)

### c)

### d)
#### 1)

### 2)

## Task 3
### a)

### b)
#### 1)

#### 2)

#### 3)

### c)
#### 1)

#### 2)

#### 3)

### d)